In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import re
import string
import nltk
import re
import string
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer


In [56]:
# text processing
def text_processing(sentence):
    # stop words are the most common words in a language
    stop_words = set(nltk.corpus.stopwords.words('english'))

    # stemming is the process for reducing devived words to their stem
    # ex: owned -> owe
    stemmer = PorterStemmer()

    # Lemmatization of words
    # grouping together the different inflects forms of a word
    # so they can be analysed as a single item
    lemmatizer = WordNetLemmatizer()
    # remove number
    sentence = re.sub(r'\d+', '', sentence)
    # remove punctuations
    sentence = sentence.translate(sentence.maketrans("","", string.punctuation))
    words_in_sentence = list(set(sentence.split(' ')) - stop_words)
    
    for index, word in enumerate(words_in_sentence):
        word = stemmer.stem(word)
        words_in_sentence[index] = lemmatizer.lemmatize(word)
    return words_in_sentence

In [57]:
data = pd.read_csv('data/bbc-text.csv')
text = data['text'].values.tolist()
category = data['category'].values.tolist()
classes = list(set(category))
category = data['category'].apply(classes.index).values.tolist()

In [58]:
clean_text = []
for i in range(len(text)):
    temp = text_processing(text[i])
    del temp[0]
    clean_text.append(''.join(temp)) 
x_train, x_test, y_train, y_test = train_test_split(clean_text, category, test_size=0.2, random_state=42)

In [59]:
from collections import Counter
print(Counter(y_test))

Counter({1: 101, 4: 98, 3: 83, 2: 82, 0: 81})


In [60]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix


nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(x_train, y_train)

from sklearn.metrics import classification_report
y_pred = nb.predict(x_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(confusion_matrix(y_test, y_pred))



accuracy 0.2808988764044944
[[ 3  0  0  0 78]
 [ 0  5  0  0 96]
 [ 0  0 14  0 68]
 [ 0  0  0  5 78]
 [ 0  0  0  0 98]]
